In [ ]:
# 第 9 章 符合Python风格的对象
"""
得益于 Python 数据模型，自定义类型的行为可以像内置类型那样自然。 实现如此自然的行为，靠的不是继承，而是鸭子类型(duck typing): 
我们只需按照预定行为实现对象所需的方法即可。

前一章分析了很多内置对象的结构和行为，这一章则自己定义类，而且 让类的行为跟真正的 Python 对象一样。

这一章接续第 1 章，说明如何实现在很多 Python 类型中常见的特殊方 法。

本章包含以下话题:
  支持用于生成对象其他表示形式的内置函数(如repr()、bytes()，等等) 
  使用一个类方法实现备选构造方法
  扩展内置的 format() 函数和 str.format() 方法使用的格式微语 言
  实现只读属性
  把对象变为可散列的，以便在集合中及作为 dict 的键使用 
  利用 __slots__ 节省内存
   
我们将开发一个简单的二维欧几里得向量类型，在这个过程中涵盖上述全部话题。 在实现这个类型的中间阶段，我们会讨论两个概念:
  如何以及何时使用 @classmethod 和 @staticmethod 装饰器
  Python 的私有属性和受保护属性的用法、约定和局限 
  
我们从对象表示形式函数开始。
"""

In [ ]:
# 9.1 对象表示形式

"""
每门面向对象的语言至少都有一种获取对象的字符串表示形式的标准方式。Python 提供了两种方式。

repr()
  以便于开发者理解的方式返回对象的字符串表示形式。
str()
  以便于用户理解的方式返回对象的字符串表示形式。
  
正如你所知，我们要实现 __repr__ 和 __str__ 特殊方法，为 repr() 和 str() 提供支持。
为了给对象提供其他的表示形式，还会用到另外两个特殊方 法:__bytes__ 和 __format__。
__bytes__ 方法与 __str__ 方法类 似:bytes() 函数调用它获取对象的字节序列表示形式。
而 __format__ 方法会被内置的 format() 函数和 str.format() 方法调 用，使用特殊的格式代码显示对象的字符串表示形式。
我们将在下一个 示例中讨论 __bytes__ 方法，随后再讨论 __format__ 方法。
"""

In [4]:
# 9.2 再谈向量类
"""
为了说明用于生成对象表示形式的众多方法，我们将使用一个 Vector2d 类，
它与第 1 章中的类似。这一节和接下来的几节会不断实 现这个类。我们期望 Vector2d 实例具有的基本行为如示例 9-1 所示。
"""
# 示例 9-1 Vector2d 实例有多种表示形式  # 这个是运行效果。获得此效果前，需要先运行 示例 9-2
v1 = Vector2d(3, 4)  # Vector2d 实例的分量可以直接通过属性访问(无需调用读值方 法)。
print(v1.x, v1.y)

3.0 4.0


In [3]:
# 示例 9-2 vector2d_v0.py:目前定义的都是特殊方法
from array import array
import math


class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))

In [6]:
x, y = v1   # Vector2d 实例可以拆包成变量元组。
x, y

(3.0, 4.0)

In [7]:
v1          #  repr 函数调用 Vector2d 实例，得到的结果类似于构建实例的源码。

Vector2d(3.0, 4.0)

In [8]:
v1_clone = eval(repr(v1))  # 这里使用 eval 函数，表明 repr 函数调用 Vector2d 实例得到的是对构造方法的准确表述
v1 == v1_clone             #  Vector2d 实例支持使用 == 比较;这样便于测试。

True

In [9]:
print(v1)                  # print 函数会调用 str 函数，对 Vector2d 来说，输出的是一个有序对 

(3.0, 4.0)


In [10]:
octets = bytes(v1)         #  bytes 函数会调用 __bytes__ 方法，生成实例的二进制表示形式
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [11]:
abs(v1)                    # abs 函数会调用 __abs__ 方法，返回 Vector2d 实例的模。

5.0

In [12]:
 bool(v1), bool(Vector2d(0, 0))   # bool 函数会调用 __bool__ 方法，如果 Vector2d 实例的模为零， 返回 False，否则返回 True。

(True, False)

In [13]:
# 9.4 classmethod与staticmethod
# 示例 9-4 比较 classmethod 和 staticmethod 的行为
class Demo:
    @classmethod
    def klassmeth(*args):
        return args
    
    @staticmethod
    def statmeth(*args):
        return args
    
Demo.klassmeth()

(__main__.Demo,)

In [14]:
Demo.klassmeth('spam')

(__main__.Demo, 'spam')

In [15]:
Demo.statmeth()

()

In [16]:
Demo.statmeth('spam')

('spam',)

In [17]:
# 9.5 格式化显示
"""
内置的 format() 函数和 str.format() 方法把各个类型的格式化方式 委托给相应的 .__format__(format_spec) 方法。format_spec 是格 式说明符，
它是:
format(my_obj, format_spec) 的第二个参数，
或者 str.format() 方法的格式字符串，{} 里代换字段中冒号后面的部分
"""
br1 = 1/2.43  # BRL 到 USD 的货币兑换比价
br1

0.4115226337448559

In [18]:
format(br1, '0.4f')  # 格式说明

'0.4115'

In [20]:
'1 BRL = {rate:0.2f} USD'.format(rate=br1)

'1 BRL = 0.41 USD'

In [21]:
"""
格式规范微语言为一些内置类型提供了专用的表示代码。
比如，
b 和 x 分别表示二进制和十六进制的 int 类型，
f 表示小数形式的 float 类 型，
而 % 表示百分数形式:
"""
format(42, 'b')

'101010'

In [22]:
format(2/3, '.1%')

'66.7%'

In [23]:
"""
格式规范微语言是可扩展的，因为各个类可以自行决定如何解释 format_spec 参数。
例如， datetime 模块中的类，它们的 __format__ 方法使用的格式代码与 strftime() 函数一样。
下面是内 置的 format() 函数和 str.format() 方法的几个示例:
"""
from datetime import datetime
now = datetime.now()
format(now, '%H:%M:%S')

'06:53:12'

In [24]:
"It's now {:%I:%M %P}".format(now)

"It's now 06:53 am"

In [25]:
"""
如果类没有定义 __format__ 方法，从 object 继承的方法会返回 str(my_object)。我们为 Vector2d 类定义了 __str__ 方法，因此可 以这样做:
"""
v1 = Vector2d(3, 4)
format(v1)

'(3.0, 4.0)'

In [26]:
# 然而，如果传入格式说明符，object.__format__ 方法会抛出 TypeError:
format(v1, '.3f')

TypeError: unsupported format string passed to Vector2d.__format__

In [ ]:
"""
我们将实现自己的微语言来解决这个问题。首先，假设用户提供的格式 说明符是用于格式化向量中各个浮点数分量的。我们想达到的效果是:
"""
>>> v1 = Vector2d(3, 4) >>> format(v1)
'(3.0, 4.0)'
>>> format(v1, '.2f') '(3.00, 4.00)'
>>> format(v1, '.3e') '(3.000e+00, 4.000e+00)'

In [ ]:
"""
实现这种输出的 __format__ 方法如示例 9-5 所示。 
示例 9-5 Vector2d.__format__ 方法，第 1 版
"""
# 在Vector2d类中定义
def __format__(self, fmt_spec=''):
    components = (format(c, fmt_spec) for c in self) # ➊ 
    return '({}, {})'.format(*components) # ➋

In [30]:
# 对象能否支持 hash，即 hash(xxx)， 需要实现 该对象的 __hash__方法。

# 示例 9-9 vector2d_v3.py:完整版

from array import array 
import math

class Vector2d:
    __slots__ = ('__x', '__y')
    typecode = 'd'
    
    def __init__(self, x, y):
        self.__x = float(x) 
        self.__y = float(y)
    @property 
    def x(self):
        return self.__x
    
    @property 
    def y(self):
        return self.__y

    def __iter__(self): 
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self): 
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''): 
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1] 
            coords = (abs(self), self.angle()) 
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
            
        components = (format(c, fmt_spec) for c in coords) 
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode) 
        return cls(*memv)

In [31]:
# 9.7 Python的私有属性和“受保护的”属性
"""
Python 不能像 Java 那样使用 private 修饰符创建私有属性，但是Python 有个简单的机制，能避免子类意外覆盖“私有”属性。

举个例子。有人编写了一个名为 Dog 的类，这个类的内部用到了 mood 实例属性，但是没有将其开放。
现在，你创建了 Dog 类的子 类:Beagle。如果你在毫不知情的情况下又创建了名为 mood 的实例属 性，那么在继承的方法中就会把 Dog 类的 mood 属性覆盖掉。
这是个难 以调试的问题。

为了避免这种情况，如果以 __mood 的形式(两个前导下划线，尾部没 有或最多有一个下划线)命名实例属性，
Python 会把属性名存入实例的 __dict__ 属性中，而且会在前面加上一个下划线和类名。
因此，对 Dog 类来说，__mood 会变成 _Dog__mood;对 Beagle 类来说，会变成 _Beagle__mood。
这个语言特性叫名称改写(name mangling)。
"""
v1 = Vector2d(3, 4)
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

In [32]:
v1._Vector2d__x

3.0

In [ ]:
"""名称改写是一种安全措施，不能保证万无一失:它的目的是避免意外访 问，不能防止故意做错事"""

In [ ]:
# 9.8 使用 __slots__ 类属性节省空间
"""
默认情况下，Python 在各个实例中名为 __dict__ 的字典里存储实例属 性。如 3.9.3 节所述，为了使用底层的散列表提升访问速度，字典会消 耗大量内存。
如果要处理数百万个属性不多的实例，通过 __slots__ 类属性，能节省大量内存，方法是让解释器在元组中存储实例属性，而 不用字典。

20200604 15:20 注：从python3.6 开始字典支持有序之后，内存消耗量较以往减少了点
"""

In [34]:
"""
定义 __slots__ 的方式是，创建一个类属性，使用 __slots__ 这个名 字，并把它的值设为一个字符串构成的可迭代对象，其中各个元素表示 各个实例属性。
我喜欢使用元组，因为这样定义的 __slots__ 中所含 的信息不会变化，如示例 9-11 所示。
"""

# 示例 9-11 vector2d_v3_slots.py:只在 Vector2d 类中添加了 __slots__ 属性
from array import array 
import math

class Vector2d:
    __slots__ = ('__x', '__y')   # <===   添加了这一行
    typecode = 'd'
    
    def __init__(self, x, y):
        self.__x = float(x) 
        self.__y = float(y)
    @property 
    def x(self):
        return self.__x
    
    @property 
    def y(self):
        return self.__y

    def __iter__(self): 
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self): 
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''): 
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1] 
            coords = (abs(self), self.angle()) 
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
            
        components = (format(c, fmt_spec) for c in coords) 
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode) 
        return cls(*memv)

In [ ]:
"""
在类中定义 __slots__ 属性的目的是告诉解释器:“这个类中的所有实 例属性都在这儿了!”
这样，Python 会在各个实例中使用类似元组的结 构存储实例变量，
从而避免使用消耗内存的 __dict__ 属性。
如果有数 百万个实例同时活动，这样做能节省大量内存。

在本书提供的示例 9-12 中，百万级别的实例运行时， 使用指定 __slots__ 的方法可以只使用 1/3 内存实现原有字典的方法。
"""

"""
在类中定义 __slots__ 属性之后，实例不能再有 __slots__ 中所列名称之外的其他属性。这只是一个副作用，不是 __slots__ 存在的真正原因。
不要使用 __slots__ 属性禁止类的 用户新增实例属性。__slots__ 是用于优化的，不是为了约束程序 员。

然而，“节省的内存也可能被再次吃掉”:如果把 '__dict__' 这个名称 添加到 __slots__ 中，实例会在元组中保存各个实例的属性，
此外还 支持动态创建属性，这些属性存储在常规的 __dict__ 中。当然，把 '__dict__' 添加到 __slots__ 中可能完全违背了初衷，这取决于各个
实例的静态属性和动态属性的数量及其用法。粗心的优化甚至比提早优 化还糟糕。
此外，还有一个实例属性可能需要注意，即 __weakref__ 属性，为了 让对象支持弱引用(参见 8.6 节)，必须有这个属性。
用户定义的类中 默认就有 __weakref__ 属性。可是，如果类中定义了 __slots__ 属 性，而且想把实例作为弱引用的目标，
那么要把 '__weakref__' 添加 到 __slots__ 中。
综上，__slots__ 属性有些需要注意的地方，而且不能滥用，不能使用 它限制用户能赋值的属性。处理列表数据时 __slots__ 属性最有用，
例如模式固定的数据库记录，以及特大型数据集。然而，如果你经常处 理大量数据，一定要了解一下 NumPy(http://www.numpy.org);
此外， 数据分析库 pandas(http://pandas.pydata.org)也值得了解，这个库可以 处理非数值数据，而且能导入 / 导出很多不同的列表数据格式。
"""

In [ ]:
"""

__slots__ 的问题
总之，如果使用得当，__slots__ 能显著节省内存，不过有几点要注意。
    每个子类都要定义 __slots__ 属性，因为解释器会忽略继承的__slots__ 属性。
    实例只能拥有 __slots__ 中列出的属性，除非把 '__dict__' 加 入 __slots__ 中(这样做就失去了节省内存的功效)。
    如果不把 '__weakref__' 加入 __slots__，实例就不能作为弱引 用的目标。

如果你的程序不用处理数百万个实例，或许不值得费劲去创建不寻常的 类，那就禁止它创建动态属性或者不支持弱引用。
与其他优化措施一 样，仅当权衡当下的需求并仔细搜集资料后证明确实有必要时，才应该 使用 __slots__ 属性。

"""

In [ ]:
# 9.9 覆盖类属性